In [ ]:
import redis
import json

# Define connection variables
host = 'redis-13857.c14.us-east-1-2.ec2.redns.redis-cloud.com'
port = 13857
password = 'okNaqK7r5dKE6stSis0FlUuiDlhH2t7M'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Real-time Fraud Detection

## Description of the use case
This use case involves detecting fraudulent transactions in real-time by using Redis Bloom Filters and Streams. By analyzing transaction data, businesses can identify and prevent fraud quickly.

## Redis Data Structures Used
- **Bloom Filters**: To check for known fraudulent entities.
- **Streams**: To capture and process real-time transaction data.
- **Hashes**: To store transaction details.


In [ ]:
# Mock data for the use case
transactions = {
    'tx_1001': {'user_id': 'user_1', 'amount': 500, 'status': 'pending'},
    'tx_1002': {'user_id': 'user_2', 'amount': 3000, 'status': 'pending'},
    'tx_1003': {'user_id': 'user_3', 'amount': 700, 'status': 'pending'}
}

# Use Hashes to store transaction details
for tx_id, tx_info in transactions.items():
    r.hset(tx_id, mapping=tx_info)

# Initialize Bloom Filter for known fraudulent users
r.execute_command('BF.RESERVE', 'fraudulent_users', 0.01, 1000)
fraudulent_users = ['user_2', 'user_4', 'user_6']
for user in fraudulent_users:
    r.bf().add('fraudulent_users', user)

# Use Streams to capture real-time transaction data
r.xadd('transactions', {'tx_id': 'tx_1001', 'user_id': 'user_1', 'amount': 500})
r.xadd('transactions', {'tx_id': 'tx_1002', 'user_id': 'user_2', 'amount': 3000})
r.xadd('transactions', {'tx_id': 'tx_1003', 'user_id': 'user_3', 'amount': 3000})

# Function to process transactions and detect fraud
def process_transactions():
    stream_entries = r.xrange('transactions')
    for entry in stream_entries:
        tx_id = entry[1]['tx_id']
        user_id = entry[1]['user_id']
        amount = int(entry[1]['amount'])
        if not r.bf().exists('fraudulent_users', user_id):
            r.hset(tx_id, 'status', 'approved')
            print(f"Transaction {tx_id} is approved.")
        else:
            r.hset(tx_id, 'status', 'fraudulent')
            print(f"Transaction {tx_id} is marked as fraudulent.") 

# Process the transactions
process_transactions()

# Print the updated transaction details
for tx_id in transactions.keys():
    print(f"{tx_id}: {r.hgetall(tx_id)}")
